In [3]:
%run functions.ipynb
import multiprocessing as mp
import os
import random
import numpy as np

In [3]:
def optimize_mp(population, x, n, function, 
         min_or_max, end_optim, p_mutate,
                shared_list, result_list):
         
    best_individual_scores = []
    iteration = 0
    while True:
        iteration += 1
         
        if min_or_max == 'max':
            reverse=True
        elif min_or_max == 'min':
            reverse=False
        else:
            raise Exception('min_or_max parameter invalid. Type "min" or "max".')
        
        #Krzyżowanie i mutacja
        for i in range(0, x, 2):
            crossing_result = crossing(population[i], population[i+1], n)
            for embryon in crossing_result:
                population.append(mutate(embryon,p_mutate))
        
        #Ocena
        scores = []
        for individual in population:
            score = function(individual)
            scores.append(score)
        
        # Co 20 iteracji weź najlepszych osobników od innych procesów
        if shared_list and iteration%20==1:
            for i in range(len(shared_list)):
                if os.getpid() not in shared_list[i][0]:
                    population.append(shared_list[i][1])
                    scores.append(shared_list[i][2])
                    # Wrzucam id procesu, żeby już wiecej nie pobrać tego osobnika
                    shared_list[i][0].append(os.getpid())
        
        population_score = sorted(zip(population, scores), key=lambda x:x[1], reverse=reverse)        
        best_individual_gen = population_score[0][0]
        best_individual_score = population_score[0][1]
        best_individual_scores.append(best_individual_score)
        
        # Co 20 iteracji zapisz najlepszego osobnika na wspólną listę
        # Lista id procesów na początku, żeby nie duplikować osobników tej samej populacji
        if iteration % 20 == 0:
            if shared_list: # Jeżeli lista nie jest pusta, to dodaje osobnika tylko gdy jest lepszy
                best_shared_score = shared_list[len(shared_list)-1][2]
                if ( 
                     (min_or_max == 'max' and best_individual_score > best_shared_score) or 
                     (min_or_max == 'min' and best_individual_score < best_shared_score)
                   ):
                    shared_list.append([manager.list([os.getpid()]),best_individual_gen, 
                                best_individual_score])
            else:
                shared_list.append([manager.list([os.getpid()]),best_individual_gen, 
                                best_individual_score])
         
        sorted_population, _ = zip(*population_score)
        population = list(sorted_population[:x])
        
        if iteration%50==0:
            print(f'Iteration: {iteration} | Best score: {best_individual_score} | Process: {os.getpid()}')
        
        # zakończ jeśli nie ma poprawy w 100 ostatnich iteracjach (?)
        if len(set(best_individual_scores[-end_optim:])) == 1 and iteration > 10:
            result_list.append((best_individual_score, best_individual_gen))
            break
            

In [54]:
manager = mp.Manager()
shared_list = manager.list()
result_list = manager.list()

pop1 = initial_pop_generation(10,4,False,-40,40)
pop2 = initial_pop_generation(10,4,False,-40,40)

# pop1 = [np.array([10,9]), np.array([20,19])]
# pop2 = [np.array([5,4]), np.array([4,3])]
        
p1 = mp.Process(target=optimize_mp, 
                args=[pop1, 10, 4, func2, 
                      'max',100, 0.2,
                      shared_list,result_list])
p2 = mp.Process(target=optimize_mp, 
                args=[pop2, 10, 4, func2, 
                      'max',100, 0.2,
                      shared_list,result_list])

p1.start()
p2.start()

p1.join()
p2.join()

In [17]:
def run_mp_alg(manager, x,n,function,
               min_or_max,end_optim=100, p_mutate=0.2,p_num=mp.cpu_count(),
               start_wth_zeros=False, min_value_n=None,
                           max_value_n=None):
    '''
    manager : multiprocessing.Manager
        umożliwia przekazywanie wyników pomiędzy procesami
    x : int
        liczba osobników w populacji
    n : int
        liczba genów każdego osobnika
    function : function
        funkcja oceny osobnika
    min_or_max : String
        'min' jeżeli wartość funkcji oceny ma być zminimalizowana
        'max' jeżeli maksymalizowana
    p_mutate : float
        pradopodobieństwo mutacji; dommyślnie równe 0.2
    p_num : int
        liczba procesów; domyślnie równa liczbie CPU
    start_wth_zeros : bool
        True - początkowe cechy osobnika równe zero
        False - losowo dobrane cechy początkowe
    min_value_n : int
        dolne ogarniczenie wartości genu
    max_value_n : int
        górne ograniczenie wartości genu
    end_optim  : int
        ilość iteracji po których brak poprawy najlepszego osobnika 
        zakończy działanie algorytmu
    '''
    shared_list = manager.list()
    result_list = manager.list()
    
    populations = [initial_pop_generation(x,n,start_wth_zeros,min_value_n,max_value_n) 
                   for i in range(p_num)]
    processes = [mp.Process(target=optimize_mp, 
                                  args=[populations[i], x, n, function, 
                                  min_or_max,end_optim, p_mutate,
                                  shared_list,result_list]) for i in range(p_num)]
        
    for process in processes:
        process.start()

    for process in processes:
        process.join()
    
    if min_or_max == 'max':
        result = sorted(result_list, key=lambda x:x[0], reverse=True)[0] 
    else:
        result = sorted(result_list, key=lambda x:x[0], reverse=False)[0]
    print(result)

In [4]:
# Wielkość populacji
x = 100
# Liczba genów osobnika
n = 100
# Minimalna wartość genu
min_value_n = -30
# Maksymalna wartość genu
max_value_n = 30
# Inicjowanie osobnika zerami (w przypadku tych zadan nie ma sensu)
start_wth_zeros = False
# Pradopodobieństwo mutacjmanager = mp.Manager()i
p_mutate = 0.2
# Funkcja oceny
function = func2
# Cel (min/max)
min_or_max='max'
# Zakończ, jeśli wynik najlepszego osobnika nie zmienił się od ... iteracji.
end_optim=100
# Liczba procesów
p_num=8
# Utworzenie managera procesów
manager = mp.Manager()

run_mp_alg(manager,x,n,function,min_or_max,end_optim,p_mutate,p_num,start_wth_zeros,min_value_n,max_value_n)

Iteration: 50 | Best score: -16.033214686261903 | Process: 6235
Iteration: 50 | Best score: -15.775635076662713 | Process: 6233
Iteration: 50 | Best score: -15.423145548956747 | Process: 6244
Iteration: 50 | Best score: -15.269987484248938 | Process: 6248Iteration: 50 | Best score: -14.97643927305262 | Process: 6242

Iteration: 50 | Best score: -14.94867350732234 | Process: 6257
Iteration: 50 | Best score: -14.831854667849694 | Process: 6239
Iteration: 50 | Best score: -14.980073069583426 | Process: 6259
Iteration: 100 | Best score: -11.936666330698976 | Process: 6235
Iteration: 100 | Best score: -11.628185632917141 | Process: 6233
Iteration: 100 | Best score: -10.784586630526938 | Process: 6248
Iteration: 100 | Best score: -10.116251268255565 | Process: 6244Iteration: 100 | Best score: -10.144200801687065 | Process: 6257

Iteration: 100 | Best score: -9.926833102966974 | Process: 6242
Iteration: 100 | Best score: -9.590288537939678 | Process: 6239
Iteration: 100 | Best score: -9.54863